<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-function-inputs" data-toc-modified-id="Get-function-inputs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get function inputs</a></span></li><li><span><a href="#Break-Down-Thermal-Funtions" data-toc-modified-id="Break-Down-Thermal-Funtions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Break Down Thermal Funtions</a></span></li><li><span><a href="#Cuda/CuPy-for-Tridiagonal-Piece-of-Code" data-toc-modified-id="Cuda/CuPy-for-Tridiagonal-Piece-of-Code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Cuda/CuPy for Tridiagonal Piece of Code</a></span></li></ul></div>

In [ ]:
import numpy as np
#import cupy as cp
#import nvtx
import pickle as pk
#import os 
#os.environ['picaso_refdata'] = '/nobackup/nbatalh1/reference_data/picaso/reference'
#os.environ['PYSYN_CDBS'] ='/nobackup/nbatalh1/reference_data/grp/hst/cdbs'
#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi

#from numba import vectorize, float64,int32,float32,int64
#from numba import guvectorize,jit

## Get function inputs

In [ ]:
opacity = jdi.opannection(
#filename_db='/nobackup/nbatalh1/reference_data/picaso/reference/opacities/all_opacities_0.6_6_R60000.db'
)#wave_range=[0.3,1]) #We can use wave_range to control the number of wave points
start_case = jdi.inputs()
#phase angle 
start_case.phase_angle(0) #radians

#define gravity
start_case.gravity(radius=1, radius_unit=jdi.u.Unit('R_jup'),
                   mass=1, mass_unit=jdi.u.Unit('M_jup'),
                  ) #any astropy units available
start_case.inputs['surface_reflect'] = 0 
start_case.inputs['hard_surface'] = 0 
#define star 
start_case.star(opacity, 5000,0,4.0, radius=1, radius_unit=jdi.u.Unit('R_sun')) #opacity db, pysynphot database, temp, metallicity, logg 
start_case.atmosphere(filename=jdi.jupiter_pt(), delim_whitespace=True)
reflected_input, thermal_input, transmission_input = jdi.gpu_test(start_case, opacity)

## Break Down Thermal Funtions

In [ ]:
from picaso.fluxes import pre_get_thermal_1d, tri_get_thermal_1d, post_get_thermal_1d

In [ ]:
for_tri,pre_out = pre_get_thermal_1d(*thermal_input)
tri_out=tri_get_thermal_1d(*for_tri)
thermal = post_get_thermal_1d(pre_out, tri_out)

## Cuda/CuPy for Tridiagonal Piece of Code 

In [ ]:
def  tri_get_thermal_1d(A, B, C, D,nlayer, nwno):#input for_tri

    positive = zeros((nlayer, nwno))
    negative = zeros((nlayer, nwno))

    #========================= Start loop over wavelength =========================
    L = nlayer+nlayer
    for w in range(nwno):
        #coefficient of posive and negative exponential terms 
        X = tri_diag_solve(L, A[:,w], B[:,w], C[:,w], D[:,w])
        #unmix the coefficients
        positive[:,w] = X[::2] + X[1::2] #Y1+Y2 in toon (table 3)
        negative[:,w] = X[::2] - X[1::2] #Y1-Y2 in toon (table 3)

    return positive, negative

#@jit(nopython=True, cache=True)
def tri_diag_solve(l, a, b, c, d):
    AS, DS, CS, DS,XK = zeros(l), zeros(l), zeros(l), zeros(l), zeros(l) # copy arrays

    AS[-1] = a[-1]/b[-1]
    DS[-1] = d[-1]/b[-1]

    for i in range(l-2, -1, -1):
        x = 1.0 / (b[i] - c[i] * AS[i+1])
        AS[i] = a[i] * x
        DS[i] = (d[i]-c[i] * DS[i+1]) * x
    XK[0] = DS[0]
    for i in range(1,l):
        XK[i] = DS[i] - AS[i] * XK[i-1]
    return XK